# Importar las librerias necesarias para la ejecusión del proyecto

In [1]:
import pandas as pd
import boto3
from dotenv import load_dotenv
import os
from lifelines import CoxPHFitter

KeyboardInterrupt: 

# Descarga los datos desde el componente de S3 en AWS

In [ ]:

load_dotenv('../.env')
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')

# Parámetros del bucket
bucket_name = 'assessment-86fc5eb8'
key = 'raw-data/data.csv'
local_path = '../data/data.csv'

# Crear carpeta
os.makedirs(os.path.dirname(local_path), exist_ok=True)


# Sesión con credenciales explícitas
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name='us-east-1' 
)

s3 = session.client('s3')

# Descargar archivo
s3.download_file(bucket_name, key, local_path)

ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden

# Importar los datos del proyecto

In [ ]:
local_path = '../data/data.csv'
df = pd.read_csv(local_path)
df.head(10)

,fecha_compra,usuario,producto,cantidad,precio
0,2014-10-01,6cba0c,PROD-efd583,2,164.7
1,2014-10-01,6cba0c,PROD-a726f5,2,164.7
2,2014-10-01,6cba0c,PROD-3e1cd9,2,164.7
3,2014-10-01,6cba0c,PROD-d012c0,2,164.7
4,2014-10-01,6cba0c,PROD-48924a,3,164.7
5,2014-10-01,6cba0c,PROD-4c2559,2,164.7
6,2014-10-01,6cba0c,PROD-10e913,2,164.7
7,2014-10-01,6cba0c,PROD-3065f9,2,164.7
8,2014-10-01,6cba0c,PROD-f47365,2,164.7
9,2014-10-01,6cba0c,PROD-e658d1,1,164.7


In [ ]:
num_usuarios=len(df['usuario'].unique())
num_productos=len(df['producto'].unique())
print(f"Número de usuarios únicos : {num_usuarios} y número de productos únicos: {num_productos}")

Número de usuarios únicos : 197392 y número de productos únicos: 16854


# Paso 1: Identificar los clientes recurrentes del supermercado

#### Los clientes recurrentes deben cumplir esta condicion. Un cliente es recurrente si:

Hace más de una compra.
(Es decir, el cliente ha comprado al menos dos veces.)

Cada compra tiene más de 10 productos.
(En cada compra que cuenta, la cantidad total de productos debe ser mayor a 10.)

Las compras ocurren dentro de períodos de 30 días.
(Entre una compra y la siguiente, pasan 30 días o menos.)

In [ ]:
# Convertir fecha a datetime
df['fecha_compra'] = pd.to_datetime(df['fecha_compra'])

# Agrupar por usuario y fecha, sumar productos comprados
compras = df.groupby(['usuario', 'fecha_compra']).agg(
    productos_comprados=('cantidad', 'sum')
).reset_index()

# Filtrar compras con más de 10 productos
compras_validas = compras[compras['productos_comprados'] > 10].copy()

# Ordenar por usuario y fecha
compras_validas = compras_validas.sort_values(['usuario', 'fecha_compra'])
compras_validas.head(10)

,usuario,fecha_compra,productos_comprados
0,000019,2014-10-26,42
1,000019,2014-12-24,25
3,00009c,2014-12-16,47
8,000166,2014-11-08,12
9,000166,2014-11-25,37
11,000342,2014-10-31,14
12,000342,2014-12-06,16
15,000481,2014-11-15,113
16,00065b,2014-10-29,157
18,0006ec,2014-12-28,82


Identificar los clientes que hacen más de una compra

In [ ]:
# Contar número de compras válidas por usuario
conteo_compras = compras_validas.groupby('usuario').size().reset_index(name='num_compras_validas')

# Filtrar solo usuarios con más de una compra válida
usuarios_mas_de_una_compra = conteo_compras[conteo_compras['num_compras_validas'] > 1]

# Unir esa información a la tabla de compras válidas
compras_recurrentes = compras_validas.merge(usuarios_mas_de_una_compra, on='usuario')

# Resultado
print(compras_recurrentes.head())

  usuario fecha_compra  productos_comprados  num_compras_validas
0  000019   2014-10-26                   42                    2
1  000019   2014-12-24                   25                    2
2  000166   2014-11-08                   12                    2
3  000166   2014-11-25                   37                    2
4  000342   2014-10-31                   14                    2


Por información ¿Cuántos de estos clientes me realizan más de dos compras? 

In [ ]:
# Contar número de compras válidas por usuario
conteo_compras = compras_validas.groupby('usuario').size().reset_index(name='num_compras_validas')

# Filtrar usuarios con más de 2 compras válidas
usuarios_mas_de_dos_compras = conteo_compras[conteo_compras['num_compras_validas'] > 2]

# Mostrar cuántos son
print(f"Clientes con más de 2 compras válidas: {len(usuarios_mas_de_dos_compras)}")

Clientes con más de 2 compras válidas: 52814


Identificar de los clientes cuáles compran en un intervalo de 30 dias 

In [ ]:
# 1. Calcular dif_dias
compras_validas['dif_dias'] = compras_validas.groupby('usuario')['fecha_compra'].diff().dt.days

# 2. Filtrar usuarios con más de una compra
multi_compra = compras_validas.groupby('usuario').filter(lambda g: len(g) > 1)

# 3. Filtrar compras donde alguna diferencia de días es <= 30
mask = (
    multi_compra
    .groupby('usuario')['dif_dias']
    .transform(lambda s: s.dropna().le(30).any())
)
compras_recurrentes_30d = multi_compra[mask].copy()

# 4. Usuarios únicos resultantes
usuarios_recurrentes = compras_recurrentes_30d['usuario'].unique()

print(f"Clientes recurrentes (compras en ≤ 30 días): {len(usuarios_recurrentes)}")
print(compras_recurrentes_30d.head())

Clientes recurrentes (compras en ≤ 30 días): 65448
   usuario fecha_compra  productos_comprados  dif_dias
8   000166   2014-11-08                   12       NaN
9   000166   2014-11-25                   37      17.0
40  000998   2014-12-25                   31       NaN
41  000998   2014-12-28                   56       3.0
44  000a09   2014-10-07                   42       NaN


Revisar las compras de un del cliente "000166"

In [ ]:
cliente = '000166'
info_000166 = compras_recurrentes_30d[
    compras_recurrentes_30d['usuario'] == cliente
]

# Filtrar columnas clave del usuario que estoy revisando
print(info_000166[['usuario', 'fecha_compra', 'productos_comprados', 'dif_dias']])

  usuario fecha_compra  productos_comprados  dif_dias
8  000166   2014-11-08                   12       NaN
9  000166   2014-11-25                   37      17.0


# Objetivo: Construir una tabla RFM (Recency–Frequency–Monetary): 

#Técnica de segmentación de clientes que se utiliza para identificar a los clientes más valiosos del supermercado. 
Recency (R): ¿Qué tan recientemente compró el cliente?
Frequency (F): ¿Con qué frecuencia compra el cliente?
Monetary (M): ¿Cuánto gasta el cliente en total o en promedio?

In [ ]:
compras_recurrentes_30d = compras_recurrentes_30d.sort_values(['usuario', 'fecha_compra'])

# 2. Se calcula la frecuencia y montos acumulados por usuario hasta cada fecha_compra
def calc_cumulative_counts(df):
    return df.groupby('usuario').cumcount() + 1  # frequency acumulado

def calc_cumulative_monetary(df):
    return df.groupby('usuario')['productos_comprados'].cumsum()  # monetary acumulado

compras_recurrentes_30d['frequency_dynamic'] = calc_cumulative_counts(compras_recurrentes_30d)
compras_recurrentes_30d['monetary_dynamic'] = calc_cumulative_monetary(compras_recurrentes_30d)

# 3. Para cada fila, recency es la diferencia en días entre la fecha_compra de esa fila y la última compra previa
# Calculamos la última fecha de compra anterior a la actual (shift por usuario)
compras_recurrentes_30d['last_purchase_prior'] = (
    compras_recurrentes_30d
    .groupby('usuario')['fecha_compra']
    .shift(1)
)

# Recency dinámico es la diferencia entre la fecha de la fila y la última compra previa
compras_recurrentes_30d['recency_dynamic'] = (
    (compras_recurrentes_30d['fecha_compra'] - compras_recurrentes_30d['last_purchase_prior'])
    .dt.days
)

# Nota: La primera compra por usuario tendrá recency_dynamic = NaN (porque no hay compra anterior)

compras_recurrentes_30d['recency_dynamic'] = compras_recurrentes_30d['recency_dynamic'].fillna(0)

# Historial acumulado 
print(compras_recurrentes_30d[['usuario', 'fecha_compra', 'frequency_dynamic', 'monetary_dynamic', 'recency_dynamic']].head(10))


   usuario fecha_compra  frequency_dynamic  monetary_dynamic  recency_dynamic
8   000166   2014-11-08                  1                12              0.0
9   000166   2014-11-25                  2                49             17.0
40  000998   2014-12-25                  1                31              0.0
41  000998   2014-12-28                  2                87              3.0
44  000a09   2014-10-07                  1                42              0.0
45  000a09   2014-11-27                  2                97             51.0
46  000a09   2014-12-04                  3               149              7.0
52  000b65   2014-10-08                  1                35              0.0
53  000b65   2014-10-16                  2                48              8.0
54  000b65   2014-10-25                  3                76              9.0


Anexar a la base de datos la variable mes

In [ ]:
# Convertir fecha_compra a datetime
compras_recurrentes_30d['fecha_compra'] = pd.to_datetime(compras_recurrentes_30d['fecha_compra'])

# Crear columna 'mes' 
compras_recurrentes_30d['mes'] = compras_recurrentes_30d['fecha_compra'].dt.month

# Verificar resultado
print(compras_recurrentes_30d[['usuario', 'fecha_compra', 'mes', 'frequency_dynamic', 'monetary_dynamic', 'recency_dynamic']].head(10))

# Renombrar la base de datos 
base_enriquecida = compras_recurrentes_30d.copy()


   usuario fecha_compra  mes  frequency_dynamic  monetary_dynamic  \
8   000166   2014-11-08   11                  1                12   
9   000166   2014-11-25   11                  2                49   
40  000998   2014-12-25   12                  1                31   
41  000998   2014-12-28   12                  2                87   
44  000a09   2014-10-07   10                  1                42   
45  000a09   2014-11-27   11                  2                97   
46  000a09   2014-12-04   12                  3               149   
52  000b65   2014-10-08   10                  1                35   
53  000b65   2014-10-16   10                  2                48   
54  000b65   2014-10-25   10                  3                76   

    recency_dynamic  
8               0.0  
9              17.0  
40              0.0  
41              3.0  
44              0.0  
45             51.0  
46              7.0  
52              0.0  
53              8.0  
54              9.0 

# Objetivo: Construcción de modelos para predecir: Cuándo los clientes volveran a comprar

Preparación de datos

In [ ]:
# Ordenar por usuario y fecha
base_enriquecida = base_enriquecida.sort_values(['usuario', 'fecha_compra'])

# Crear columna con la fecha de la siguiente compra para cada usuario (shift negativo)
base_enriquecida['fecha_siguiente_compra'] = base_enriquecida.groupby('usuario')['fecha_compra'].shift(-1)

# Calcular duración entre compras (en días)
base_enriquecida['duration'] = (base_enriquecida['fecha_siguiente_compra'] - base_enriquecida['fecha_compra']).dt.days

# El evento ocurre si existe fecha_siguiente_compra (1), o está censurado (0) si no hay siguiente compra
base_enriquecida['event'] = base_enriquecida['fecha_siguiente_compra'].notnull().astype(int)

# Para la última compra sin siguiente
# Duration: diferencia entre fecha_compra y fecha_ref (fecha máxima de observación)
fecha_ref = base_enriquecida['fecha_compra'].max()
mask_censura = base_enriquecida['event'] == 0
base_enriquecida.loc[mask_censura, 'duration'] = (fecha_ref - base_enriquecida.loc[mask_censura, 'fecha_compra']).dt.days

# Eliminar filas con duración negativa o cero
base_enriquecida = base_enriquecida[base_enriquecida['duration'] > 0]

print(base_enriquecida[['usuario', 'fecha_compra', 'fecha_siguiente_compra', 'duration', 'event']].head(10))


   usuario fecha_compra fecha_siguiente_compra  duration  event
8   000166   2014-11-08             2014-11-25      17.0      1
9   000166   2014-11-25                    NaT      33.0      0
40  000998   2014-12-25             2014-12-28       3.0      1
44  000a09   2014-10-07             2014-11-27      51.0      1
45  000a09   2014-11-27             2014-12-04       7.0      1
46  000a09   2014-12-04                    NaT      24.0      0
52  000b65   2014-10-08             2014-10-16       8.0      1
53  000b65   2014-10-16             2014-10-25       9.0      1
54  000b65   2014-10-25             2014-10-31       6.0      1
55  000b65   2014-10-31             2014-11-19      19.0      1


In [ ]:
base_enriquecida.head()

,usuario,fecha_compra,productos_comprados,dif_dias,frequency_dynamic,monetary_dynamic,last_purchase_prior,recency_dynamic,mes,fecha_siguiente_compra,duration,event
8,000166,2014-11-08,12,NaN,1,12,NaT,0.0,11,2014-11-25,17.0,1
9,000166,2014-11-25,37,17.0,2,49,2014-11-08,17.0,11,NaT,33.0,0
40,000998,2014-12-25,31,NaN,1,31,NaT,0.0,12,2014-12-28,3.0,1
44,000a09,2014-10-07,42,NaN,1,42,NaT,0.0,10,2014-11-27,51.0,1
45,000a09,2014-11-27,55,51.0,2,97,2014-10-07,51.0,11,2014-12-04,7.0,1


# Modelos de supervivencia

In [ ]:
# Realizar una copia de la base original 
base_supervivencia = base_enriquecida.copy()

# Confirmar que la copia fue creada
print(base_supervivencia.shape)
print(base_supervivencia.head())

(361232, 12)
   usuario fecha_compra  productos_comprados  dif_dias  frequency_dynamic  \
8   000166   2014-11-08                   12       NaN                  1   
9   000166   2014-11-25                   37      17.0                  2   
40  000998   2014-12-25                   31       NaN                  1   
44  000a09   2014-10-07                   42       NaN                  1   
45  000a09   2014-11-27                   55      51.0                  2   

    monetary_dynamic last_purchase_prior  recency_dynamic  mes  \
8                 12                 NaT              0.0   11   
9                 49          2014-11-08             17.0   11   
40                31                 NaT              0.0   12   
44                42                 NaT              0.0   10   
45                97          2014-10-07             51.0   11   

   fecha_siguiente_compra  duration  event  
8              2014-11-25      17.0      1  
9                     NaT      33.0  

Ajustar el modelo de supervivencia

In [ ]:
# Inicializar modelo CoxPH
cph = CoxPHFitter()

# Seleccionar columnas para el modelo (incluye duration y event)
cols_modelo = ['duration', 'event', 'recency_dynamic', 'frequency_dynamic', 'monetary_dynamic', 'mes']

# Ajustar modelo con los datos
cph.fit(base_supervivencia[cols_modelo], duration_col='duration', event_col='event')

# Mostrar resumen con coeficientes, p-values y tests
cph.print_summary()


<lifelines.CoxPHFitter: fitted with 361232 total observations, 61121 right-censored observations>
             duration col = 'duration'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 361232
number of events observed = 300111
   partial log-likelihood = -3539840.91
         time fit was run = 2025-05-26 16:05:30 UTC

---
                   coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                          
recency_dynamic   -0.02      0.98      0.00           -0.02           -0.02                0.98                0.98
frequency_dynamic  0.10      1.10      0.00            0.10            0.10                1.10                1.10
monetary_dynamic  -0.00      1.00      0.00           -0.00           -0.00                1.00                1.00
mes               -0.16      0.85      0.00           -0.17           -0.16                0.85                0.86

                   cmp to      z      p  -log2(p)
covariate                                        
recency_dynamic      0.00 -88.40 <0.005       inf
frequency_dynamic    0.00 210.23 <0.005       inf
monetary_dynamic     0.00 -15.31 <0.005    173.39
mes                  0.00 -55.39 <0.005       inf
---
Concordance = 0.66
Partial AIC = 7079689.83
log-likelihood ratio test = 68781.38 on 4 df
-log2(p) of ll-ratio test = inf

In [ ]:
# Crear base de predicción con las variables necesarias
X_pred = base_supervivencia[['recency_dynamic', 'frequency_dynamic', 'monetary_dynamic', 'mes']]

: 

In [ ]:
# 1. Predecir días esperados hasta próxima compra
base_supervivencia['dias_hasta_proxima_compra'] = cph.predict_expectation(X_pred)

# 2. Asegurar que 'fecha_compra' esté en formato datetime
base_supervivencia['fecha_compra'] = pd.to_datetime(base_supervivencia['fecha_compra'])

# 3. Calcular fecha estimada próxima compra sumando días esperados
base_supervivencia['fecha_proxima_compra'] = base_supervivencia['fecha_compra'] + pd.to_timedelta(base_supervivencia['dias_hasta_proxima_compra'], unit='D')

# 4. Mostrar las columnas relevantes para verificar
print(base_supervivencia[['usuario', 'fecha_compra', 'dias_hasta_proxima_compra', 'fecha_proxima_compra']].head(10))


In [ ]:
# Ordenamos por usuario y fecha_compra para que la primera fila sea la compra más reciente
base_supervivencia = base_supervivencia.sort_values(['usuario', 'fecha_compra'], ascending=[True, False])

# Seleccionamos solo la fila más reciente por usuario
base_ultima_compra = base_supervivencia.groupby('usuario').first().reset_index()

# Mostramos el resultado final
print(base_ultima_compra[['usuario', 'fecha_compra', 'dias_hasta_proxima_compra', 'fecha_proxima_compra']].head(10))


  usuario fecha_compra  dias_hasta_proxima_compra  \
0  000166   2014-11-25                  22.136895   
1  000998   2014-12-25                  19.973480   
2  000a09   2014-12-04                  19.347749   
3  000b65   2014-12-22                  12.338302   
4  000c7d   2014-11-01                  20.377711   
5  000db9   2014-12-25                   7.018267   
6  000dee   2014-12-27                  17.454387   
7  0010a3   2014-12-25                   5.065570   
8  0010a5   2014-12-16                  34.713619   
9  0011b0   2014-11-17                  28.203318   

           fecha_proxima_compra  
0 2014-12-17 03:17:07.702950816  
1 2015-01-13 23:21:48.690891480  
2 2014-12-23 08:20:45.521089359  
3 2015-01-03 08:07:09.333462820  
4 2014-11-21 09:03:54.249511645  
5 2015-01-01 00:26:18.263279100  
6 2015-01-13 10:54:19.042894500  
7 2014-12-30 01:34:25.243214805  
8 2015-01-19 17:07:36.724265019  
9 2014-12-15 04:52:46.695379307  


In [ ]:
base_ultima_compra.head(5)

,usuario,fecha_compra,productos_comprados,dif_dias,frequency_dynamic,monetary_dynamic,last_purchase_prior,recency_dynamic,mes,fecha_siguiente_compra,duration,event,dias_hasta_proxima_compra,fecha_proxima_compra
0,000166,2014-11-25,37,17.0,2,49,2014-11-08,17.0,11,2014-11-25,33.0,0,22.136895,2014-12-17 03:17:07.702950816
1,000998,2014-12-25,31,NaN,1,31,NaT,0.0,12,2014-12-28,3.0,1,19.973480,2015-01-13 23:21:48.690891480
2,000a09,2014-12-04,52,7.0,3,149,2014-11-27,7.0,12,2014-12-04,24.0,0,19.347749,2014-12-23 08:20:45.521089359
3,000b65,2014-12-22,46,10.0,8,264,2014-12-12,10.0,12,2014-12-22,6.0,0,12.338302,2015-01-03 08:07:09.333462820
4,000c7d,2014-11-01,63,13.0,2,77,2014-10-19,13.0,11,2014-11-01,57.0,0,20.377711,2014-11-21 09:03:54.249511645


# Predecir los 3 productos más probables (baseline sencillo)

In [ ]:
# Agrupar productos por usuario y sumar las cantidades
productos_por_usuario = (
    df.groupby(['usuario', 'producto'])['cantidad']
    .sum()
    .reset_index()
)

# Ordenar productos por usuario según cantidad comprada descendente
productos_por_usuario = productos_por_usuario.sort_values(['usuario', 'cantidad'], ascending=[True, False])

# Crear lista con los productos ordenados por cantidad para cada usuario
productos_ordenados = (
    productos_por_usuario.groupby('usuario')['producto']
    .apply(list)
    .reset_index(name='productos_ordenados')
)


In [ ]:
# Tomar solo los 3 primeros productos por usuario
productos_ordenados['top_3_productos'] = productos_ordenados['productos_ordenados'].apply(lambda x: x[:3])

In [ ]:
# Crear base de datos con los productos
base_con_productos = base_enriquecida.merge(
    productos_ordenados[['usuario', 'top_3_productos']],
    on='usuario',
    how='left'
)

# Ahora base_con_productos tiene la columna 'top_3_productos' con la predicción para cada usuario


In [ ]:
print(base_con_productos[['usuario', 'top_3_productos']].head())

  usuario                          top_3_productos
0  000166  [PROD-62e82f, PROD-0256a8, PROD-4d5e04]
1  000166  [PROD-62e82f, PROD-0256a8, PROD-4d5e04]
2  000998  [PROD-74efc1, PROD-dab6fc, PROD-5eb939]
3  000a09  [PROD-3bea74, PROD-412320, PROD-66a332]
4  000a09  [PROD-3bea74, PROD-412320, PROD-66a332]


Salida

In [ ]:
# Se toma la información estimada por el modelo y los productos top para cada usuario
base_ultima_compra[['usuario', 'fecha_proxima_compra']].head()
base_con_productos[['usuario', 'top_3_productos']].head()

# Unir las tablas por la columna "usuario"
base_final = base_ultima_compra.merge(
    base_con_productos[['usuario', 'top_3_productos']],
    on='usuario',
    how='left'  # Mantiene todos los registros de base_ultima_compra
)

# Paso 3: Verifica el resultado
base_final[["usuario","fecha_proxima_compra","top_3_productos"]].head()


,usuario,fecha_proxima_compra,top_3_productos
0,000166,2014-12-17 03:17:07.702950816,"[PROD-62e82f, PROD-0256a8, PROD-4d5e04]"
1,000166,2014-12-17 03:17:07.702950816,"[PROD-62e82f, PROD-0256a8, PROD-4d5e04]"
2,000998,2015-01-13 23:21:48.690891480,"[PROD-74efc1, PROD-dab6fc, PROD-5eb939]"
3,000a09,2014-12-23 08:20:45.521089359,"[PROD-3bea74, PROD-412320, PROD-66a332]"
4,000a09,2014-12-23 08:20:45.521089359,"[PROD-3bea74, PROD-412320, PROD-66a332]"


# Guardar la salida en un .parquet

In [ ]:
os.makedirs('cleaned-data/TRM550', exist_ok=True)
base_final.to_parquet('cleaned-data/TRM550/resultados_usuarios.parquet', index=False)
print("✅ Archivo guardado en cleaned-data/TRM550/resultados_usuarios.parquet")

✅ Archivo guardado en cleaned-data/TRM550/resultados_usuarios.parquet


# Subir el archivo a S3

In [ ]:
bucket_name = 'assessment-86fc5eb8'
ruta_s3 = 'cleaned-data/TRM550/resultados_usuarios.parquet'

s3.upload_file('cleaned-data/TRM550/resultados_usuarios.parquet', bucket_name, ruta_s3)
print(f"✅ Archivo subido exitosamente a s3://{bucket_name}/{ruta_s3}")

✅ Archivo subido exitosamente a s3://assessment-86fc5eb8/cleaned-data/TRM550/resultados_usuarios.parquet


# Verificar si el archivo se subio a S3

In [ ]:
# Parámetros del bucket y archivo
bucket_name = 'assessment-86fc5eb8'
key = 'cleaned-data/TRM550/resultados_usuarios.parquet'
local_path = 'resultados_usuarios_descargado.parquet'

# Sesión con credenciales explícitas
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name='us-east-1'
)
s3 = session.client('s3')

# Descargar archivo desde S3
s3.download_file(bucket_name, key, local_path)
print(f"✅ Archivo descargado como {local_path}")

# Comprobar que el archivo existe localmente
if os.path.exists(local_path):
    print("✅ El archivo fue descargado correctamente y existe en el sistema de archivos local.")
else:
    print("❌ El archivo no se encuentra en el sistema de archivos local.")

✅ Archivo descargado como resultados_usuarios_descargado.parquet
✅ El archivo fue descargado correctamente y existe en el sistema de archivos local.
